### Load Pretrained model ResNet50 to check categories of GIF

In [1]:
# Import libraries
import os
import io
import pandas as pd
import requests
from PIL import Image
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
# Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

In [3]:
def download_image(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return Image.open(io.BytesIO(response.content))
    except Exception as e:
        print(f"Error downloading image from {url}: {str(e)}")
        return None


In [4]:
# Predict the categories of each gif images

def categorize_gif(img):
    try:
        # Convert to RGB if necessary
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Resize image to 224x224 (ResNet50 input size)
        img = img.resize((224, 224))
        
        # Convert image to array and preprocess
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        # Make prediction
        predictions = model.predict(img_array)
        
        # Decode and return top 3 predicted categories
        return decode_predictions(predictions, top=3)[0]
    
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return None

In [5]:
def main():
    #tsv_file = 'tgif-v1.0.tsv'
    
    # Read the TSV file
   
    try:
        df = pd.read_csv('tgif-v1.0.tsv', sep='\t', header=None, names=['url', 'description'])
    except Exception as e:
        print(f"Error reading TSV file: {str(e)}")
        return
 

    # Assuming the column containing URLs is named 'url'
    # If it's named differently, replace 'url' with the correct column name
    if 'url' not in df.columns:
        print("Error: The TSV file does not contain a 'url' column.")
        return

    for index, row in df.iterrows():
        url = row['url']
        print(f"\nProcessing: {url}")
        
        img = download_image(url)
        if img:
            categories = categorize_gif(img)
            if categories:
                print("Top 3 categories:")
                for i, (imagenet_id, label, score) in enumerate(categories, 1):
                    print(f"{i}. {label} ({score:.2f})")
            else:
                print("Unable to categorize this GIF.")
        else:
            print("Unable to download or process this GIF.")

if __name__ == "__main__":
    main()


Processing: https://38.media.tumblr.com/9f6c25cc350f12aa74a7dc386a5c4985/tumblr_mevmyaKtDf1rgvhr8o1_500.gif
1/1 [==============================] - 3s 3s/step
Top 3 categories:
1. paintbrush (0.12)
2. suit (0.08)
3. mask (0.07)

Processing: https://38.media.tumblr.com/9ead028ef62004ef6ac2b92e52edd210/tumblr_nok4eeONTv1s2yegdo1_400.gif
1/1 [==============================] - 0s 218ms/step
Top 3 categories:
1. Persian_cat (0.32)
2. chow (0.31)
3. Pomeranian (0.20)

Processing: https://38.media.tumblr.com/9f43dc410be85b1159d1f42663d811d7/tumblr_mllh01J96X1s9npefo1_250.gif
1/1 [==============================] - 0s 226ms/step
Top 3 categories:
1. chain_mail (0.34)
2. breastplate (0.30)
3. shield (0.15)

Processing: https://38.media.tumblr.com/9f659499c8754e40cf3f7ac21d08dae6/tumblr_nqlr0rn8ox1r2r0koo1_400.gif
1/1 [==============================] - 0s 202ms/step
Top 3 categories:
1. gorilla (1.00)
2. chimpanzee (0.00)
3. baboon (0.00)

Processing: https://38.media.tumblr.com/9ed1c99afa7d71411

KeyboardInterrupt: 